In [1]:
import orca
import warnings

import urbansim_templates
import os; os.chdir('../')

from scripts import models, datasources, variables

warnings.simplefilter('ignore')

Registering model step 'TOD_choice'
Registering model step 'WLCM'


In [2]:
import pandas as pd
import numpy as np

### Generate accessibility variables (or load them if they already exist)

Initialize networks

In [3]:
orca.run(['initialize_network_small', 'initialize_network_walk'])

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


If they exist already:

In [4]:
# walk_net_vars = pd.read_csv('../ual_model_workspace/fall-2018-models/data/walk_net_vars.csv', index_col='osmid')
# drive_net_vars = pd.read_csv('../ual_model_workspace/fall-2018-models/data/drive_net_vars.csv', index_col='osmid')
# orca.add_table('nodeswalk', walk_net_vars)
# orca.add_table('nodessmall', drive_net_vars)

If generating on-the-fly:

In [5]:
orca.run(['network_aggregations_small', 'network_aggregations_walk'])

### Simulate workplace location choices

Run the model step

In [6]:
orca.run(['wlcm_simulate'])

Running step 'wlcm_simulate'
Iteration 1: 191308 of 3060996 valid choices
Iteration 2: 381885 of 3060996 valid choices
Iteration 3: 571544 of 3060996 valid choices
Iteration 4: 760216 of 3060996 valid choices
Iteration 5: 947624 of 3060996 valid choices
Iteration 6: 1133740 of 3060996 valid choices
Iteration 7: 1318175 of 3060996 valid choices
Iteration 8: 1500174 of 3060996 valid choices
Iteration 9: 1679647 of 3060996 valid choices
Iteration 10: 1855438 of 3060996 valid choices
Iteration 11: 2025064 of 3060996 valid choices
Iteration 12: 2186620 of 3060996 valid choices
Iteration 13: 2335048 of 3060996 valid choices
Iteration 14: 2459339 of 3060996 valid choices
Iteration 15: 2540512 of 3060996 valid choices
Iteration 16: 2573848 of 3060996 valid choices
Iteration 17: 2578045 of 3060996 valid choices
Iteration 18: 2578046 of 3060996 valid choices
No valid alternatives for the remaining choosers
Time to execute step 'wlcm_simulate': 242.11 s
Total time to execute iteration 1 with iter

Query the updated persons table to assess job placement

In [7]:
persons_w_jobs = orca.get_table('persons').to_frame()
num_persons_w_jobs = len(persons_w_jobs[~pd.isnull(persons_w_jobs['job_id'])])
wlcm = urbansim_templates.modelmanager.get_step('WLCM')
num_workers = len(persons_w_jobs.query(' and '.join(wlcm.out_chooser_filters)))

In [8]:
print("WLCM placed {0} of {1} workers ({2}%) during simulation.".format(
    num_persons_w_jobs, num_workers, np.round(num_persons_w_jobs/num_workers * 100, 1)))

WLCM placed 2578046 of 3060996 workers (84.2%) during simulation.


### Simulate time-of-day choices for mandatory trip (commute) departure times

#### a) Assign commuters to depature time chunks

In [9]:
orca.run(['TOD_choice_simulate'])

Running step 'TOD_choice_simulate'
Time to execute step 'TOD_choice_simulate': 226.53 s
Total time to execute iteration 1 with iteration value None: 226.53 s


#### b) Assign actual departure times based on known distributions

In [10]:
orca.run(['TOD_distribution_simulate'])

Running step 'TOD_distribution_simulate'
Time to execute step 'TOD_distribution_simulate': 615.06 s
Total time to execute iteration 1 with iteration value None: 615.06 s


Query the updated persons table to assess TOD chunk assignment

In [11]:
persons_w_TOD = orca.get_table('persons').to_frame()
num_persons_w_TOD = len(persons_w_TOD[~pd.isnull(persons_w_TOD['HW_ST'])])

In [12]:
print("Departure time assigned to {0} of {1} workers with jobs ({2}%) during simulation.".format(
    num_persons_w_TOD, num_workers, np.round(num_persons_w_TOD/num_persons_w_jobs * 100, 1)))

Departure time assigned to 2502338 of 3060996 workers with jobs (97.1%) during simulation.


### Generate activity plans

Activity plans currently written according to [BEAM](http://beam.lbl.gov/) specification

In [13]:
orca.run(['generate_activity_plans'])

Running step 'generate_activity_plans'
Time to execute step 'generate_activity_plans': 132.94 s
Total time to execute iteration 1 with iteration value None: 132.94 s


Load activity plan data and investigate it

In [14]:
activities = pd.read_csv('./data/urbansim_beam_plans.csv')

In [15]:
activities.head()

,personId,planElement,planElementIndex,activityType,x,y,endTime
0,0,activity,1,Home,-121.882277,37.339126,8.609233
1,0,leg,2,NaN,NaN,NaN,NaN
2,0,activity,3,Work,-122.054273,37.398092,15.724806
3,0,leg,4,NaN,NaN,NaN,NaN
4,0,activity,5,Home,-121.882277,37.339126,NaN
